In [1]:
from pricing_library.market import Market
from pricing_library.option import Option
from pricing_library.trinomial_tree import TrinomialTree

from datetime import datetime

t = TrinomialTree(
    market=Market(
        interest_rate=0.04,
        volatility=0.25,
        spot_price=100,
        dividend_price=0,
        dividend_ex_date=datetime(2024, 5, 24),
    ),
    pricing_date=datetime(2023, 9, 20),
    n_steps=1,
)
price = t.price(
    opt=Option(
        option_type="call",
        exercise_type="eu",
        strike_price=102,
        maturity_date=datetime(2024, 9, 19),
    ),
    # draw_tree=True
)

Elapsed time the __build_tree function: 0.01 seconds
Elapsed time the price function: 0.01 seconds


In [2]:
price


9.541798983778728

In [3]:
t.root


Node<spot price: 100.00, next mid price: : 104.08, next up price: 160.48, next down price: 67.50, variance: 698.66, p down: 0.21, p mid: 0.66, p up: 0.13, current date: 2023-09-20 00:00:00, option value: 9.54>

In [4]:
t

TrinomialTree<1 steps, delta_t: 1.000, alpha: 1.542, root: Node<spot price: 100.00, next mid price: : 104.08, next up price: 160.48, next down price: 67.50, variance: 698.66, p down: 0.21, p mid: 0.66, p up: 0.13, current date: 2023-09-20 00:00:00, option value: 9.54
 next mid node: Node<spot price: 104.08, next mid price: : 108.33, next up price: 167.03, next down price: 70.26, variance: 756.85, p down: 0.21, p mid: 0.66, p up: 0.13, current date: 2024-09-19 00:00:00, option value: 2.08>
 next lower node: Node<spot price: 67.50, next mid price: : 70.26, next up price: 108.33, next down price: 45.57, variance: 318.35, p down: 0.21, p mid: 0.66, p up: 0.13, current date: 2024-09-19 00:00:00, option value: 0.00>
 next upper node: Node<spot price: 160.48, next mid price: : 167.03, next up price: 257.55, next down price: 108.33, variance: 1799.37, p down: 0.21, p mid: 0.66, p up: 0.13, current date: 2024-09-19 00:00:00, option value: 58.48>>>

In [5]:
# params = {"n": 2000,
#           "s0": 100,
#           "vol": 0.25,
#           "rate": 0.04,
#           "strike": 102,
#           "maturity": datetime.date(year=2024, month=9, day=19),
#           "t0": datetime.date(year=2023, month=9, day=20),
#           "market": "european",
#           "div": 0
#           }